# *Notebook preparation*

In [ ]:
## RUN ONLY IF ON GOOGLE COLAB

# Load the Drive helper and mount

from google.colab import drive

# This will prompt for authorization

drive.mount('/content/gdrive/')

In [ ]:
#!pip uninstall googletrans
#!pip install googletrans
#!pip install googletrans-temp

In [ ]:
#!pip uninstall googletrans
#!git clone https://github.com/BoseCorp/py-googletrans.git
#!cd ./py-googletrans
#!python setup.py install

In [ ]:
#pip install polyglot

In [ ]:
#!apt install python-numpy libicu-dev
#!apt install python3-icu
#!pip3 install pycld2

In [ ]:
#!pip3 install morfessor

In [ ]:
#!polyglot download sentiment2.en

In [ ]:
#!polyglot download sentiment2.pt

In [ ]:
#!polyglot download embeddings2.en

In [ ]:
#!polyglot download ner2.en

In [ ]:
#!pip install -U textblob
#!python -m textblob.download_corpora

In [2]:
!pip install numpy
!pip install scikit-surprise

In [3]:
!pip install tabulate

In [4]:
!pip install vaderSentiment

In [5]:
#!pip install translate

In [6]:
# Import libraries

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import importlib 
import googletrans 
importlib.reload(googletrans)
from googletrans import Translator
#from polyglot.downloader import downloader
#from polyglot.text import Text
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from translate import Translator

import time
import datetime
import random

import six
from tabulate import tabulate

from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import accuracy

In [6]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.7.6.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
machine          : AMD64
processor        : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.0.1
numpy            : 1.18.1
pytz             : 2019.3
dateutil         : 2.8.1
pip              : 20.0.2
setuptools       : 45.2.0.post20200210
Cython           : 0.29.15
pytest           : 5.3.5
hypothesis       : 5.5.4
sphinx           : 2.4.0
blosc            : None
feather          : None
xlsxwriter       : 1.2.7
lxml.etree       : 4.5.0
html5lib         : 1.0.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.1
IPython          : 7.12.0
pandas_datareader: None
bs4              : 4.8.2
bottleneck       : 1.3.2
fastparquet      : None
gcsfs            : None
lxml.etree       : 

# *Data acquisition*

In [7]:
# Load csvs

customers = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_customers_dataset.csv')
order_items = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_order_items_dataset.csv')
products_en = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/product_category_name_translation.csv')
sellers = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_sellers_dataset.csv')
products = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_products_dataset.csv')
orders = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_orders_dataset.csv')
reviews = pd.read_csv('C:/Users/SSICI/Documents/UCL/Dissertation/data/olist_order_reviews_dataset.csv')

# *Data exploration*

In [ ]:
## Data exploration

# List dataframes

dfs = [customers, order_items, products_en, products, sellers, orders, reviews]

# Columns names and df shape

for df in dfs:
    print(df.shape, '\n', df.columns, '\n')

In [ ]:
# Dataframes heads

customers.head()

In [ ]:
order_items.head()

In [ ]:
products_en

In [ ]:
sellers.head()

In [ ]:
products.head()

In [ ]:
orders.head()

In [ ]:
reviews.head()

In [ ]:
# Review scores

sns.set(color_codes = True)

x = reviews.review_score
sns.distplot(x, bins = 5, kde = False, rug = False)

In [ ]:
# Average review score

print('Average review score is ', np.mean(x))

In [ ]:
# Unique customers

print('Number of unique customer is ', len(customers.customer_unique_id.unique()))

In [ ]:
# Unique products

print('Number of unique products is ', len(products.product_id.unique()))

In [ ]:
# Unique customers in orders table

print('Number of unique customers in orders table is ', len(orders.customer_id.unique()))

In [ ]:
# Customers by state

customers.customer_state.value_counts(normalize = True) * 100

In [ ]:
# Sellers by state

sellers.seller_state.value_counts(normalize = True) * 100

In [ ]:
# Customers by city

customers.customer_city.value_counts(normalize = True) * 100

In [ ]:
# Sellers by city

sellers.seller_city.value_counts(normalize = True) * 100

In [ ]:
# Customers by zip code

customers.customer_zip_code_prefix.value_counts(normalize = True) * 100

In [ ]:
# Sellers by zip code

sellers.seller_zip_code_prefix.value_counts(normalize = True) * 100

In [ ]:
# Average # orders by customer

len(customers.customer_id.unique()) / len(customers.customer_unique_id.unique())

In [ ]:
# Customers ranking by order of users

customers.customer_unique_id.value_counts()

In [ ]:
# % orders by product

order_items.product_id.value_counts(normalize = True) * 100

## Data Wrangling

In [8]:
# Add english category name to product dataframe

products = products.join(products_en.set_index('product_category_name'), on = 'product_category_name', how = 'left')
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [9]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,order+item_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,00010242fe8c5a6d1ba2dd792cb162141
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,00018f77f2f0320c557190d7a144bdd31
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,000229ec398224ef6ca0657da4fc703e1
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery,00024acbcdf0a6daa1e931b038114c751
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools,00042b26cf59d7ce69dfabb4e55b4fd91


In [10]:
# Add product info on order dataframe

order_items = order_items.drop(columns = ['product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english'])

order_items = order_items.join(products.set_index('product_id'), on = 'product_id', how = 'left')
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order+item_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools


In [11]:
# English category names of products # orders

product_categories_counts = order_items.product_category_name_english.value_counts().rename_axis('unique_values').reset_index(name='counts')
print(product_categories_counts)

                unique_values  counts
0              bed_bath_table   11115
1               health_beauty    9670
2              sports_leisure    8641
3             furniture_decor    8334
4       computers_accessories    7827
..                        ...     ...
66      arts_and_craftmanship      24
67          cds_dvds_musicals      14
68                 la_cuisine      14
69  fashion_childrens_clothes       8
70      security_and_services       2

[71 rows x 2 columns]


In [12]:
#sns.set(style = "whitegrid")

#data = product_categories_counts
#x = "unique_values"
#y = "counts"

#ax = sns.barplot(x = x, y = y, data = data)
#ax.set_xticklabels(rotation = 90)
#ax.set(xticklabels = [0])

# *New features*

In [13]:
# Add # of orders by customer

customers['orders_by_customer'] = customers['customer_unique_id'].map(customers['customer_unique_id'].value_counts())
customers['orders_by_customer'].value_counts()

1     93099
2      5490
3       609
4       120
5        40
6        36
7        21
17       17
9         9
Name: orders_by_customer, dtype: int64

In [14]:
# Add order+item_id

order_items['order+item_id'] = order_items['order_id'].map(str) + order_items['order_item_id'].map(str)
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order+item_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools


In [15]:
# Export csv

#customers.to_csv('olist_customers_dataset.csv', index = False, index_label = False)
#!cp olist_customers_dataset.csv '/content/gdrive/My Drive/data/modified/'

#order_items.to_csv('olist_order_items_dataset.csv', index = False, index_label = False)
#!cp olist_order_items_dataset.csv '/content/gdrive/My Drive/data/modified/'

# *Sentiment analysis for reviews*

In [ ]:
reviews.review_comment_title.value_counts()

In [ ]:
reviews.review_comment_message.value_counts()

### TextBlob attempt

In [ ]:
# Testing TextBlob sentiment

#blob = TextBlob('uneventful')
#blob.sentiment.polarity

In [ ]:
# Change to string type

#reviews['review_comment_message'] = reviews['review_comment_message'].apply(str)

In [ ]:
# Translation - WARNING, this takes a while...

#translator = Translator()
#reviews['review_comment_message_en'] = reviews['review_comment_message'].apply(translator.translate)

In [ ]:
# Load the comments into textblob

#desc_blob = [TextBlob(desc) for desc in reviews['review_comment_message_en']]

In [ ]:
# Add the sentiment metrics to the dataframe

#reviews['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
#reviews['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

# Show dataframe

#df.head()

In [ ]:
#reviews.tb_Pol.value_counts()

### VADER attempt

In [85]:
# Vader sandbox

analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

def polarity_score(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

polarity_score("Bank’s $3tn fund arm warns Labor department proposal could jeopardise retirement incomes")

-0.1027

In [ ]:
len(reviews_new['polarity'])

In [ ]:
#reviews_new.to_csv('reviews_new_polarity.csv')

In [ ]:
# Googletrans sandbox

#translator = Translator()

#comment_en = translator.translate('Todos os produtos vieram errados, cor diferente, móvel quebrado e não consigo pós venda nem de loja, nem email, nem telefone.')
#print(comment_en.text)

In [86]:
# Change reviews comments messages into string

reviews['review_comment_message'] = pd.Series(reviews['review_comment_message'],
                                                      dtype="string")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41614 entries, 0 to 41613
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               41614 non-null  int64  
 1   review_id                41614 non-null  object 
 2   order_id                 41614 non-null  object 
 3   review_score             41614 non-null  int64  
 4   review_comment_title     9944 non-null   object 
 5   review_comment_message   41614 non-null  string 
 6   review_creation_date     41614 non-null  object 
 7   review_answer_timestamp  41614 non-null  object 
 8   polarity                 41614 non-null  float64
 9   score_detailed           41614 non-null  float64
dtypes: float64(2), int64(2), object(5), string(1)
memory usage: 3.2+ MB


In [87]:
reviews.shape

(41614, 10)

In [104]:
# Drop NA to clean reviews comment messages column

reviews_new = reviews.replace(to_replace='None', value=np.nan).dropna(subset = ['review_comment_message'])

In [105]:
reviews_new.shape

(41614, 10)

In [90]:
reviews_new['review_comment_message'].to_frame().T.dtypes.value_counts()

string    41614
dtype: int64

In [106]:
reviews_new.head()

,Unnamed: 0,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,polarity,score_detailed
0,0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,0.000,5.000
1,1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,0.000,5.000
2,2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47,-0.296,3.704
3,3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.",2018-02-16 00:00:00,2018-02-20 10:52:22,0.000,4.000
4,4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01,0.296,5.296


In [110]:
#reviews_new = reviews_new.reset_index()
reviews_new = reviews_new.drop(columns=['index', 'Unnamed: 0'])
reviews_new.head()

,level_0,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,polarity,score_detailed
0,0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,0.000,5.000
1,1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,0.000,5.000
2,2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47,-0.296,3.704
3,3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.",2018-02-16 00:00:00,2018-02-20 10:52:22,0.000,4.000
4,4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01,0.296,5.296


In [ ]:
# Test translation

#comment_test_en = translator.translate(reviews_new.at[41752, 'review_comment_message']).text
#print(comment_test_en)

In [ ]:
#reviews_new.to_csv('reviews_new_check.csv')

In [ ]:
dirty_ids = ['532bd7f1a84d53519ee94e47a8fdd5ba',
'c90bf23aaba3fd9a2712864a04fd7ba2',
'e98c4c4fb9bd358090c29cfd94129f67',
'0c09c5e8ce5acbe3b42fda5b386f6fd9',
'c71de58300ef4e16196c1a7f55f7c288',
'3140a4eb146655fe104d54d385f9e9a0',
'ff9c1c1c60e0d0d8357dcfddd5d30c74',
'282bc4139e6dc1b0065e4508483df6f8',
'ecfbf704562a8043c8298864c0bd7587',
'72bfa12277357444b874425ec8565963',
'dd719cd14a15e23fcd73bb43dbdf74cb',
'34dee513bfdaa15ec31cbdcb739da5a2',
'e1e2bc5d0e1b38619384d1e703876dbc',
'a18feffd6107bf8510123fbfd85daf89',
'779e808c363bb3c1cd3711c56c891f50',
'563b3426e49f0b82875311d1af344e90',
'cbc036cc8c3edc20daf02f781a5f7a39',
'700d5d0b433653540b499f02c834c026',
'40a8ba42f2470c246c62bc225d8096bc',
'a011fac23fc9bcc61058ae835f40105a',
'6f9e9a580ce1d349a4f5e613928ae952',
'b9614a882b978cd0fccb5f3cfd9acdf2',
'de400a079a1f6cc20777c234cb19baed',
'5032db77926bb1be245591c4e89f0c40',
'1fe9d2ee706fb28c431aa5d933245d45',
'1ce11bd2d9c397e0b673035dcc533f5c',
'b905a83cd40f67b08363261fc2d2dadb',
'5911592a4b0c81d4936fab9aedcd6426',
'052150c42f7cc01bad8bd1b3d760f2df',
'6f3f89d587ee33fc38f4dcf439655dc6',
'4f072e1788bda93be16074632ec37314',
'e0a6fd9c545231f6d15f2bb7ea4e731b',
'11b314b40342d0d976f0aa354f99cee4',
'2b49e26af77fe3de28229362aa604b00',
'e203d589007409e506b8ac99b998b36d',
'abc06de5376f9a6ece665b74c35c8de9',
'4221f3f3b5ffcf57d8eb3497d3332479',
'b2c19dad0f5ff7f2cd1c4a1d27646820',
'479455dcf5c7b17c5025011776ac844e',
'00b385c367ca681c5800956f540f62b8',
'05c827a05cbc45e2bf60b0c4206c671d',
'007c08c01178c42ff829560cc24dd0d6',
'b47d194b6d89d7533d105e79538baf8d',
'53e4882ef92d517d3402c7c459fb0a08',
'42efe873e1d493d3a3a0e4f9b8ed7bee',
'8f965d010b3119d9962e091de3425fb9',
'c3e786a1d01515813421425a6ff24c82',
'd7d32715dd20179a35afb317c5febcd1',
'b9615ba0300186f46c31da18d0d2d910',
'90c8683fdaf866f5f52b0a5ba46b2dfb',
'26e159bbb2adde0a6d8c4d7d5a8d0265',
'bc9c92ac80dc427c68b3f3ee037a33f6',
'7f592b57c7cdf4d771713da1e7137415',
'4182970b342ec19b3fd53b67a91ee8b1',
'72449acd6059b9212ca4a214f8adbea0',
'e2e5dce3b568f4f13473c7cc9cf29554',
'a8811c3be82f060646cb8e8e99c62b15',
'a84375539025a5556aa2bea99a9455bd',
'2e9e5ca5c2c34d76c1d15347915defbe',
'93576f6bc38f8f1a213ec54f6dd79d43',
'95743d4e19eb75a413378df072ad8173',
'cf92c3a3609fb8f38d51af5740493ce6',
'd6bdcf45e23775ab2a216424fba98c32',
'5cf4cf09ef7a928bda183967caf8791a',
'b15debe453f5cdc6e292b013691b2b1d',
'c119871c16c6764cf17c8000b7ff9834',
'a8ce40a577b62789aa6120eeefe91799',
'4dbb0787588f28909efbc886f439aa59',
'4b750ee33d4b4dc252e25c37e402cc27',
'bfb0132659c358e0b8210a3f53281a49',
'aa5ab2fa61ddc4da7e3f70f1496251b0',
'd6e98db916d74dd080a119e11f77a944',
'e92c7e2c8591a747e9112eb06ca610b7',
'a1d9aa7274e2d6060279144778e95db5',
'aae14327e70d5b8e332bec53f990d370',
'41b8ac555ee326e4b90f9b21003be4d4',
'0039afc1db3af397f4f4c11cc142dce7',
'10f62fdeea5332995b98c326bc9a9ee4',
'1d623b82e18d30beb72bc080f61293fd',
'250e02784388d3d94ccef0e26fc47f3e',
'30f1cd3ec3eeec1b7f8f3c0c53524994',
'be08b76d3c2eb8a401c2af7fc8f66ff5',
'd3633c3981afb1553569164a1798d50e',
'a0ca1013e22c9a42716763d01c2bcb25',
'6d8f6f32915bd84cdcce664835d8c391',
'fd50614415226f64ae007d40b4196e8e',
'ff2ffa48dda6afd89624d4d3518ee3d6',
'fc27a4442a7e94f87a1e8b35d10a2cdc',
'b326f67288f003f81c50333c7647f723',
'4b6fdbc6f36d926407cba6c5779d233e',
'1ca7da1315f6ff270521b56eb980739a',
'6dc9c0c8d5e889435cfd771527956420',
'04bc27874dcc2bd3381806df6bd998c1',
'fe46368cddb6bdaa71b7befdc673d24c',
'ea1cc0ff71eadacfaa5543f08b70e075',
'85f45d2190062088245e06825530a6da',
'02956609af1e114830c2dc77e3a3d3f4',
'86cb15d4a00e8a6ff94f8a30af32d99b',
'64a897235b857e3e09aa49eee59d0b55',
'5e9267dff30598f8a3153f3ce0ddd6e3',
'bd595c7431cbf8f83b05bf40728d9e0f',
'7beefc8cd672c066622a9b5258bd6a81',
'0153726cf98dfa0f5c0994878e787db3',
'0820684b9ad1c78177fd0030e0a6f84c',
'1ac8fa532c9bf87b482a89d738139868',
'1a7109f64ce3f8f10ae6888714af0169',
'65796eff954d756008a2d85ba7adf86d',
'cb5cba6694706bab593637759309e268',
'9c73b71c76dc2eb8e78b980720026825',
'2ab115bf611e323b0564aa448853f41f',
'44c1e5f49fb7e0bdd58f1dd802241b6e',
'7b8b935be7d60fd808be4d40f1c23361',
'60afe2ce881d803e1240f8f80feefab7',
'5bf688550dd26337db734777ce37248d',
'533b52320e00720998fa34be5742163b',
'46f85abf15da7655c736c05898387dff',
'd02df2d850fcaa8c9974d85f0a6b13d3',
'480bc69dac211c3ae1a44533394b7d5c',
'679b7e39ac0bf1a250928e794f412739',
'3c4627e1dc671e11279e84b372eb90aa',
'f09ee5b3beb151f620472c812be87f12',
'948d72c72c13076e71d52923ecdb746b',
'b93fd94319287ea3932e92b9f6225199',
'd5ae8441ed8d821c2bfd435f3f4c9141',
'7915e960f5094fc36e0103b1592f92ba',
'b1bcc5db3418b49a915e576111f169e1',
'1acaf76c3c504b7a4e0f43a0036d5ebf',
'45b2ed2edcd0290b079318d568578f04',
'8125404d834e0d5efb9ab3bd28c8229a',
'1c88500fb94695ef3f1bbc47727d2784',
'd89c845d80493bc8047dddd39ed421a8',
'e800338d2264e19595439c36ec7a8454',
'65d0c6182ac939c3f1bd8fa7b988d84c',
'ce4410cda687ca0f9b2fe7cccab69e6d',
'65d0c6182ac939c3f1bd8fa7b988d84c',
'a1225b4f63ed96c642956130d5e2243b',
'4c97bc0ba5b4191effeff0915133181a',
'909a6dd469eabbd3877d8a43d93cfae5',
'5865ec608823234392ca9cee57f29ff9']

In [ ]:
reviews_new = reviews_new[~reviews_new['review_id'].isin(dirty_ids)]
reviews_new.shape

In [ ]:
reviews_new.head()

In [ ]:
# Translate review comments from Portuguese to English with googletrans (THIS TAKES A WHILE...)

#reviews_new['review_comment_message_en'] = reviews_new['review_comment_message'].apply(translator.translate, src='pt', dest='en').apply(getattr, args=('text',))

In [ ]:
# Translate review comments from Portuguese to English with translate (THIS TAKES A WHILE...)

#translator = Translator(from_lang = "portuguese", to_lang = "english")

#reviews_new['review_comment_message_en'] = reviews_new['review_comment_message'].apply(translator.translate)

In [ ]:
reviews_new.tail()

In [ ]:
# write output csv for memory reasons

#reviews_new.to_csv('reviews_new_en_translation.csv')

In [111]:
# VADER attempt continuation

reviews_new['polarity'] = reviews_new['review_comment_message'].apply(polarity_score)
reviews_new.tail()

,level_0,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,polarity,score_detailed
41609,41609,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,NaN,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23,0.0000,5.0000
41610,41610,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,NaN,"O produto não foi enviado com NF, não existe v...",2018-05-19 00:00:00,2018-05-20 21:51:06,-0.2960,2.7040
41611,41611,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,0.8439,5.8439
41612,41612,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43,-0.4184,0.5816
41613,41613,efe49f1d6f951dd88b51e6ccd4cc548f,90531360ecb1eec2a1fbb265a0db0508,1,NaN,"meu produto chegou e ja tenho que devolver, po...",2017-07-03 00:00:00,2017-07-03 21:01:49,0.0000,1.0000


In [ ]:
# Get polarity scores of review comments

#reviews_new['review_comment_message_polarity'] = reviews_new['review_comment_message_en'].apply(polarity_score)

In [112]:
reviews_new['polarity'].value_counts()

 0.0000    33685
-0.2960     3528
 0.5994     1016
 0.2960      378
-0.3595      308
           ...  
-0.7717        1
-0.3597        1
-0.7096        1
-0.4874        1
 0.7339        1
Name: polarity, Length: 311, dtype: int64

In [113]:
reviews_new['score_detailed'] = reviews_new['review_score'] + reviews_new['polarity']
reviews_new.head()

,level_0,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,polarity,score_detailed
0,0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,0.000,5.000
1,1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,0.000,5.000
2,2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47,-0.296,3.704
3,3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.",2018-02-16 00:00:00,2018-02-20 10:52:22,0.000,4.000
4,4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01,0.296,5.296


In [114]:
reviews_new['score_detailed'].value_counts()

5.0000    16777
1.0000     7319
4.0000     4901
3.0000     2912
2.0000     1776
          ...  
4.6247        1
0.2997        1
4.3802        1
4.1260        1
4.6166        1
Name: score_detailed, Length: 562, dtype: int64

In [ ]:
# write output csv for memory reasons

#reviews_new.to_csv('reviews_new.csv')

In [17]:
#################################################################################
# YOU CAN START FROM HERE WITH THE READY reviews_new.csv TO AVOID PROCESSING TIME
#################################################################################

reviews = pd.read_csv('reviews_new.csv')

In [18]:
reviews = reviews.drop(columns = 'Unnamed: 0')

In [19]:
# review_score vs polarity

reviews.describe()

,review_score,polarity,score_detailed
count,41614.000000,41614.000000,41614.000000
mean,3.637622,-0.004147,3.633475
std,1.627638,0.183765,1.664624
min,1.000000,-0.903900,0.096100
25%,2.000000,0.000000,2.000000
50%,4.000000,0.000000,4.296000
75%,5.000000,0.000000,5.000000
max,5.000000,0.990900,5.990900


# *Feature engineering for recommender models*

### *Content based model*

In [45]:
# Create product bag of words dataframe for similarity matrix for content based filtering

products_matrix = order_items.join(sellers.set_index('seller_id'), on = 'seller_id')
products_matrix.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order+item_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools,87900,loanda,PR


In [46]:
# Only unique product_id

products_matrix.shape
products_matrix = products_matrix.drop_duplicates(subset = 'product_id')
products_matrix.shape

(32951, 20)

In [47]:
# Drop unnecesary columns

products_matrix = products_matrix.drop(columns = ['order_id', 'order_item_id', 'shipping_limit_date', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty'])
products_matrix.shape

(32951, 14)

In [48]:
# Set product_id as index

products_matrix = products_matrix.set_index(keys = 'product_id')
products_matrix

,seller_id,price,freight_value,order+item_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state
product_id,,,,,,,,,,,,,
4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,650.0,28.0,9.0,14.0,cool_stuff,27277,volta redonda,SP
e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,30000.0,50.0,30.0,40.0,pet_shop,3471,sao paulo,SP
c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,3050.0,33.0,13.0,33.0,furniture_decor,37564,borda da mata,MG
7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,200.0,16.0,10.0,15.0,perfumery,14403,franca,SP
ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,3750.0,35.0,40.0,30.0,garden_tools,87900,loanda,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4cc4d02efc8f249c13355147fb44e34d,d1c7fa84e48cfa21a0e595167c1c500e,129.90,51.20,ffebd80e3291e811c308365936897efd1,ferramentas_jardim,6700.0,35.0,12.0,22.0,garden_tools,99730,jacutinga,RS
b10ecf8e33aaaea419a9fa860ea80fb5,0241d4d5d36f10f80c644447315af0bd,99.00,13.52,ffee31fb4b5e35c9123608015637c4951,moveis_decoracao,2300.0,37.0,30.0,20.0,furniture_decor,80330,curitiba,PR
dd469c03ad67e201bc2179ef077dcd48,7e93a43ef30c4f03f38b393420bc753a,736.00,20.91,fff7c4452f050315db1b3f24d9df5fcd1,relogios_presentes,400.0,19.0,9.0,15.0,watches_gifts,6429,barueri,SP


In [49]:
# Convert all columns to strings

products_matrix = products_matrix.astype(str)

In [50]:
products_matrix.columns

Index(['seller_id', 'price', 'freight_value', 'order+item_id',
       'product_category_name', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'seller_zip_code_prefix',
       'seller_city', 'seller_state'],
      dtype='object')

In [51]:
picky_bow_columns = ['seller_id', 'price', 'freight_value', 'product_category_name',
                    'seller_zip_code_prefix', 'seller_city', 'seller_state']

In [53]:
# Collapse bag of words columns

products_matrix['bag_of_words'] = ''
columns = picky_bow_columns
for index, row in products_matrix.iterrows():
    words = ''
    for col in columns:
      words = words + row[col]+ ' '
      row['bag_of_words'] = words
    
products_matrix.drop(columns = [col for col in products_matrix.columns if col!= 'bag_of_words'],
                     inplace = True)

products_matrix

,bag_of_words
product_id,
4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202 58.9 13.29 co...
e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36 239.9 19.93 p...
c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d 199.0 17.87 m...
7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4 12.99 12.79 p...
ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87 199.9 18.14 f...
...,...
4cc4d02efc8f249c13355147fb44e34d,d1c7fa84e48cfa21a0e595167c1c500e 129.9 51.2 fe...
b10ecf8e33aaaea419a9fa860ea80fb5,0241d4d5d36f10f80c644447315af0bd 99.0 13.52 mo...
dd469c03ad67e201bc2179ef077dcd48,7e93a43ef30c4f03f38b393420bc753a 736.0 20.91 r...


In [54]:
## This has RAM issues

# Instantiating and generating the count matrix

count = CountVectorizer()
count_matrix = count.fit_transform(products_matrix['bag_of_words'])

# Generating the cosine similarity matrix

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [55]:
cosine_sim

array([[1.        , 0.11111111, 0.        , ..., 0.11785113, 0.11785113,
        0.11785113],
       [0.11111111, 1.        , 0.        , ..., 0.11785113, 0.11785113,
        0.11785113],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.11785113, 0.11785113, 0.        , ..., 1.        , 0.125     ,
        0.125     ],
       [0.11785113, 0.11785113, 0.        , ..., 0.125     , 1.        ,
        0.125     ],
       [0.11785113, 0.11785113, 0.        , ..., 0.125     , 0.125     ,
        1.        ]])

In [56]:
# creating a Series for the products ids so they are associated to an ordered numerical
# list I will use in the function to match the indexes

indices = pd.Series(products_matrix.index)

In [57]:
#  defining the function that takes in product id 
# as input and returns the top 10 products

def recommendations(product_id, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended products
    recommended_products = []
    
    # gettin the index of the product that matches the product id
    idx = indices[indices == product_id].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 20 most similar products
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 20 matching products
    for i in top_20_indexes:
        recommended_products.append(list(products_matrix.index)[i])
        
    return recommended_products

In [58]:
# test 20 best recommendations on a product id

recommendations('bbe7651fef80287a816ead73f065fc4b')

['761673fd35e2e24ad39b5f6a96c1692c',
 'db59f0dfd5593c492b2ff1a6ab50cc2d',
 'ca36223e14b83df526098afff99de992',
 '0ae652266b7911e69661f1359912d235',
 'a7e25abccd44c762ce17be417966dec5',
 '3356d43bd57f974c4ae7fc5f9c4e589a',
 'aee1ce7ad6628e95724fb80c24479c57',
 'bfc1d1c62b1f7f401d8d9dda962bb870',
 'b89aa9d4a130c4556741ebc4a93d8844',
 'a270b2f4aad055863b643395b987d151',
 'a2e3365ef10ca17767e0228708414937',
 '42d8844b80341e62fc38c51541ff4f26',
 'e20fcfa5bcdc5c4341d6680cca2bae68',
 '8bce28781999184ac2a46c7c83ba0ed6',
 '9c0e03692735d7d179868554fc5a1d2c',
 '006baa9a5b8f95895f15273a35bc2664',
 '8ef3e9415bd7cc041d96fe685ad6dfc2',
 'f5b0aa2b1c596d29597ba3226353fce4',
 '90b7c42b2aff33b8f12c37d4c6588f99',
 '4fb5aa1b9628ff1b6e79b07001874abd']

In [59]:
# add column of recommendations to product dataset

products_matrix.reset_index(level=0, inplace=True)
products_matrix['recommendations'] = products_matrix['product_id'].map(recommendations)

In [60]:
# check head of output products_matrix with recommendations

products_matrix.head()

,product_id,bag_of_words,recommendations
0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202 58.9 13.29 co...,"[1053, 7501, 10887, 24396, 563, 25129, 13368, ..."
1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36 239.9 19.93 p...,"[12583, 21158, 22934, 24811, 6105, 1841, 31059..."
2,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d 199.0 17.87 m...,"[10500, 25026, 14000, 15089, 27699, 8032, 2376..."
3,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4 12.99 12.79 p...,"[29788, 28027, 20777, 20776, 29892, 1991, 1571..."
4,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87 199.9 18.14 f...,"[3531, 11860, 32503, 30983, 31877, 19535, 1039..."


In [61]:
# write output csv for memory reasons

products_matrix.to_csv('products_matrix_picky_bow.csv')

In [62]:
# this functions takes the list of recommendations indexes and changes it to a list of recommended
# product ids

def index_to_product_id(product_indexes):
    return list([products_matrix['product_id'][i] for i in product_indexes])

In [63]:
# apply the function to get a column with recommendations as product ids instead of indexes

products_matrix['recommended_product_ids'] = products_matrix['recommendations'].map(index_to_product_id)

In [64]:
# check head of output products_matrix with recommendations as product ids

products_matrix.head()

,product_id,bag_of_words,recommendations,recommended_product_ids
0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202 58.9 13.29 co...,"[1053, 7501, 10887, 24396, 563, 25129, 13368, ...","[3e31fd1419cc4cd6adad1925e72e3a03, 7f1a77bec06..."
1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36 239.9 19.93 p...,"[12583, 21158, 22934, 24811, 6105, 1841, 31059...","[80a28a92c0530e6838d3f35cae419d34, 7c700504b30..."
2,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d 199.0 17.87 m...,"[10500, 25026, 14000, 15089, 27699, 8032, 2376...","[a81fc5e66120ebdb295873f74a45c5b7, d52463875db..."
3,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4 12.99 12.79 p...,"[29788, 28027, 20777, 20776, 29892, 1991, 1571...","[a6553cfbe11f87ac3079d7e18ff8c87b, ff01ba3d367..."
4,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87 199.9 18.14 f...,"[3531, 11860, 32503, 30983, 31877, 19535, 1039...","[b249cf5fcf68f70e13515489f0fcb627, 5dddb31154c..."


In [65]:
# write output csv for memory reasons

products_matrix.to_csv('products_matrix_picky_bow.csv')

In [20]:
#####################################################################################
# YOU CAN START FROM HERE WITH THE READY products_matrix.csv TO AVOID PROCESSING TIME
#####################################################################################

#products_matrix = pd.read_csv('products_matrix_picky_bow.csv')

### *Collaborative filtering based model*

In [66]:
# join orders items with orders for info on customers

orders_matrix = order_items.join(orders.set_index('order_id'), on = 'order_id')
orders_matrix.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order+item_id,product_category_name,product_name_lenght,...,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,58.0,...,9.0,14.0,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,56.0,...,30.0,40.0,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,59.0,...,13.0,33.0,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,42.0,...,10.0,15.0,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,59.0,...,40.0,30.0,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [67]:
#reviews = reviews.drop(columns = ['Unnamed: 0', 'review_comment_title', 'review_comment_message',
#                        'review_creation_date', 'review_answer_timestamp', 'polarity'])

In [68]:
orders_matrix.shape

(112650, 24)

In [69]:
orders_matrix.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'order+item_id',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date'],
      dtype='object')

In [70]:
# join orders matrix with reviews for info on reviews, mainly scores

orders_matrix = orders_matrix.join(reviews.set_index('order_id'), on = 'order_id')
orders_matrix.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order+item_id,product_category_name,product_name_lenght,...,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,polarity,score_detailed
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,00010242fe8c5a6d1ba2dd792cb162141,cool_stuff,58.0,...,2017-09-20 23:43:48,2017-09-29 00:00:00,97ca439bc427b48bc1cd7177abe71365,5.0,NaN,"Perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03,0.000,5.000
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,00018f77f2f0320c557190d7a144bdd31,pet_shop,56.0,...,2017-05-12 16:04:24,2017-05-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,000229ec398224ef6ca0657da4fc703e1,moveis_decoracao,59.0,...,2018-01-22 13:19:16,2018-02-05 00:00:00,0c5b33dea94867d1ac402749e5438e8b,5.0,NaN,Chegou antes do prazo previsto e o produto sur...,2018-01-23 00:00:00,2018-01-23 16:06:31,0.000,5.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,00024acbcdf0a6daa1e931b038114c751,perfumaria,42.0,...,2018-08-14 13:32:39,2018-08-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd91,ferramentas_jardim,59.0,...,2017-03-01 16:42:31,2017-03-17 00:00:00,940144190dcba6351888cafa43f3a3a5,5.0,NaN,Gostei pois veio no prazo determinado .,2017-03-02 00:00:00,2017-03-03 10:54:59,-0.296,4.704


In [71]:
orders_matrix.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'order+item_id',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'polarity',
       'score_detailed'],
      dtype='object')

In [72]:
# drop unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['order_item_id', 'shipping_limit_date', 'price',
                                              'freight_value', 'product_category_name',
                                              'product_name_lenght', 'product_description_lenght',
                                              'product_photos_qty', 'product_weight_g',
                                              'product_length_cm', 'product_height_cm', 'product_width_cm',
                                              'product_category_name_english', 'order_status',
                                              'order_purchase_timestamp', 'order_approved_at',
                                              'order_delivered_carrier_date',
                                              'order_delivered_customer_date',
                                              'order_estimated_delivery_date', 'review_answer_timestamp'])

orders_matrix.columns

Index(['order_id', 'product_id', 'seller_id', 'order+item_id', 'customer_id',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date', 'polarity',
       'score_detailed'],
      dtype='object')

In [73]:
# save a backup dataframe of orders matrix as orders_matrix_old

#orders_matrix_old = orders_matrix

In [74]:
# drop more unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['order_id', 'seller_id', 'order+item_id',
                                              'review_score', 'review_comment_title',
                                              'review_comment_message', 'review_creation_date',
                                             'polarity'])

orders_matrix.columns

Index(['product_id', 'customer_id', 'review_id', 'score_detailed'], dtype='object')

In [75]:
# rearrange order of colums

orders_matrix = orders_matrix[['customer_id', 'product_id', 'score_detailed', 'review_id']]
orders_matrix.head()

,customer_id,product_id,score_detailed,review_id
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000,97ca439bc427b48bc1cd7177abe71365
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN,NaN
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000,0c5b33dea94867d1ac402749e5438e8b
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN,NaN
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704,940144190dcba6351888cafa43f3a3a5


In [76]:
# add recommended_product_ids as a column joined on product_id

orders_matrix = orders_matrix.join(products_matrix.set_index('product_id'), on = 'product_id')
orders_matrix.head()

,customer_id,product_id,score_detailed,review_id,bag_of_words,recommendations,recommended_product_ids
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000,97ca439bc427b48bc1cd7177abe71365,48436dade18ac8b2bce089ec2a041202 58.9 13.29 co...,"[1053, 7501, 10887, 24396, 563, 25129, 13368, ...","[3e31fd1419cc4cd6adad1925e72e3a03, 7f1a77bec06..."
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN,NaN,dd7ddc04e1b6c2c614352b383efe2d36 239.9 19.93 p...,"[12583, 21158, 22934, 24811, 6105, 1841, 31059...","[80a28a92c0530e6838d3f35cae419d34, 7c700504b30..."
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000,0c5b33dea94867d1ac402749e5438e8b,5b51032eddd242adc84c38acab88f23d 199.0 17.87 m...,"[10500, 25026, 14000, 15089, 27699, 8032, 2376...","[a81fc5e66120ebdb295873f74a45c5b7, d52463875db..."
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN,NaN,9d7a1d34a5052409006425275ba1c2b4 12.99 12.79 p...,"[29788, 28027, 20777, 20776, 29892, 1991, 1571...","[a6553cfbe11f87ac3079d7e18ff8c87b, ff01ba3d367..."
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704,940144190dcba6351888cafa43f3a3a5,df560393f3a51e74553ab94004ba5c87 199.9 18.14 f...,"[3531, 11860, 32503, 30983, 31877, 19535, 1039...","[b249cf5fcf68f70e13515489f0fcb627, 5dddb31154c..."


In [77]:
# drop unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['bag_of_words', 'recommendations'])
orders_matrix.head()

,customer_id,product_id,score_detailed,review_id,recommended_product_ids
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000,97ca439bc427b48bc1cd7177abe71365,"[3e31fd1419cc4cd6adad1925e72e3a03, 7f1a77bec06..."
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN,NaN,"[80a28a92c0530e6838d3f35cae419d34, 7c700504b30..."
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000,0c5b33dea94867d1ac402749e5438e8b,"[a81fc5e66120ebdb295873f74a45c5b7, d52463875db..."
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN,NaN,"[a6553cfbe11f87ac3079d7e18ff8c87b, ff01ba3d367..."
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704,940144190dcba6351888cafa43f3a3a5,"[b249cf5fcf68f70e13515489f0fcb627, 5dddb31154c..."


In [78]:
orders_matrix.shape

(112827, 5)

In [79]:
def correct_list(some_dirty_list):
    return some_dirty_list.replace("'","").replace("[","").replace("]","").replace(" ","").split(",")

correct_list(orders_matrix['recommended_product_ids'][0])

AttributeError: 'list' object has no attribute 'replace'

In [31]:
# correct list format

orders_matrix['recommended_product_ids'] = orders_matrix['recommended_product_ids'].apply(correct_list)
orders_matrix.head()

,customer_id,product_id,score_detailed,review_id,recommended_product_ids
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000,97ca439bc427b48bc1cd7177abe71365,"[436c8d57ff8d4aa254318e9bd9b48c83, fc0234a0ca2..."
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN,NaN,"[68935616eb789144ba60861fa5a60e3e, 7c700504b30..."
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000,0c5b33dea94867d1ac402749e5438e8b,"[d52463875db80535c279cf58f6f5c183, 928c2c0fdf4..."
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN,NaN,"[ff01ba3d3676dd02937fd007e3f4e992, a6553cfbe11..."
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704,940144190dcba6351888cafa43f3a3a5,"[b249cf5fcf68f70e13515489f0fcb627, 5dddb31154c..."


In [80]:
# explode recommended_product_ids to change it to rows

orders_matrix_augmentation = orders_matrix.explode('recommended_product_ids')
orders_matrix_augmentation.shape

(2256540, 5)

In [81]:
orders_matrix_augmentation.head()

,customer_id,product_id,score_detailed,review_id,recommended_product_ids
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.0,97ca439bc427b48bc1cd7177abe71365,3e31fd1419cc4cd6adad1925e72e3a03
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.0,97ca439bc427b48bc1cd7177abe71365,7f1a77bec0639e473392e91a504272a5
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.0,97ca439bc427b48bc1cd7177abe71365,d65b6607952f9e0d705b1cbdc92ac027
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.0,97ca439bc427b48bc1cd7177abe71365,fc0234a0ca2e8c140d56247254f8e618
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.0,97ca439bc427b48bc1cd7177abe71365,436c8d57ff8d4aa254318e9bd9b48c83


In [82]:
# change product_id to recommended_product_ids

orders_matrix_augmentation['product_id'] = orders_matrix_augmentation['recommended_product_ids']

# change review_id to 'content_based_augmented'

orders_matrix_augmentation['review_id'] = 'content_based_augmented'

# drop unnecesary columns

orders_matrix_augmentation = orders_matrix_augmentation.drop(columns = ['recommended_product_ids'])

# show head of augmentation dataframe

orders_matrix_augmentation.head()

,customer_id,product_id,score_detailed,review_id
0,3ce436f183e68e07877b285a838db11a,3e31fd1419cc4cd6adad1925e72e3a03,5.0,content_based_augmented
0,3ce436f183e68e07877b285a838db11a,7f1a77bec0639e473392e91a504272a5,5.0,content_based_augmented
0,3ce436f183e68e07877b285a838db11a,d65b6607952f9e0d705b1cbdc92ac027,5.0,content_based_augmented
0,3ce436f183e68e07877b285a838db11a,fc0234a0ca2e8c140d56247254f8e618,5.0,content_based_augmented
0,3ce436f183e68e07877b285a838db11a,436c8d57ff8d4aa254318e9bd9b48c83,5.0,content_based_augmented


In [83]:
orders_matrix_augmentation.tail()

,customer_id,product_id,score_detailed,review_id
112649,96d649da0cc4ff33bb408b199d4c7dcf,10d9c039b5dcb3b2a4a4fd507fd187a0,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,9c7da616e5b3dd8100858b72e883b7a8,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,6ebdbcab1f0dc7e1d3ecb01c1b140ade,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,7494075d75b7855aae586070f6a12113,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,c4ac3cc6a0553682784fe2a569e3df4f,NaN,content_based_augmented


In [84]:
# change to show all outputs

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [85]:
# merge original and augmentation dataframes and show all shapes for check

orders_matrix_augmented = orders_matrix.append(orders_matrix_augmentation)
orders_matrix.shape
orders_matrix_augmentation.shape
orders_matrix_augmented.shape
orders_matrix.head()

(112827, 5)

(2256540, 4)

(2369367, 5)

,customer_id,product_id,score_detailed,review_id,recommended_product_ids
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000,97ca439bc427b48bc1cd7177abe71365,"[3e31fd1419cc4cd6adad1925e72e3a03, 7f1a77bec06..."
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN,NaN,"[80a28a92c0530e6838d3f35cae419d34, 7c700504b30..."
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000,0c5b33dea94867d1ac402749e5438e8b,"[a81fc5e66120ebdb295873f74a45c5b7, d52463875db..."
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN,NaN,"[a6553cfbe11f87ac3079d7e18ff8c87b, ff01ba3d367..."
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704,940144190dcba6351888cafa43f3a3a5,"[b249cf5fcf68f70e13515489f0fcb627, 5dddb31154c..."


In [86]:
# drop unnecesary columns

orders_matrix_augmented = orders_matrix_augmented.drop(columns = ['recommended_product_ids'])
orders_matrix_augmented.tail()

,customer_id,product_id,score_detailed,review_id
112649,96d649da0cc4ff33bb408b199d4c7dcf,10d9c039b5dcb3b2a4a4fd507fd187a0,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,9c7da616e5b3dd8100858b72e883b7a8,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,6ebdbcab1f0dc7e1d3ecb01c1b140ade,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,7494075d75b7855aae586070f6a12113,NaN,content_based_augmented
112649,96d649da0cc4ff33bb408b199d4c7dcf,c4ac3cc6a0553682784fe2a569e3df4f,NaN,content_based_augmented


In [87]:
# drop unnecesary columns

orders_matrix_augmented = orders_matrix_augmented.drop(columns = ['review_id'])
orders_matrix_augmented.head()

,customer_id,product_id,score_detailed
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704


In [88]:
orders_matrix_augmented.shape

(2369367, 3)

In [89]:
# write output csv for memory reasons

orders_matrix_augmented.to_csv('order_matrix_augmented_picky_bow.csv')
#orders_matrix.to_csv('order_matrix.csv')

In [33]:
#####################################################################################
# YOU CAN START FROM HERE WITH THE READY order_matrix_augmented.csv TO AVOID PROCESSING TIME
#####################################################################################

#orders_matrix_augmented = pd.read_csv('order_matrix_augmented_picky_bow.csv')
#orders_matrix = pd.read_csv('order_matrix.csv')

In [90]:
orders_matrix_augmented.shape

(2369367, 3)

In [91]:
orders_matrix_augmented.score_detailed.isna().sum()

1352232

In [92]:
orders_matrix_augmented = orders_matrix_augmented.dropna(subset = ['score_detailed'])

In [93]:
orders_matrix_augmented.score_detailed.isna().sum()

0

In [94]:
orders_matrix_augmented.shape

(1017135, 3)

In [95]:
# set dataframe for Surprise models

reader = Reader(line_format='user item rating', sep=';', rating_scale=(0, 6), skip_lines=1)
data = Dataset.load_from_df(orders_matrix_augmented, reader = reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


# Train all algorithms

#np.random.seed(42)

#train = data.build_full_trainset()
#test = train.build_testset()

#classes = (SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline,
#           CoClustering, BaselineOnly)

#algorithms = [SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline,
#           CoClustering, BaselineOnly]

#rmses = []
#maes = []

#kf = KFold(random_state=0)

# Iterate over all algorithms
#for klass in classes:
#  out = cross_validate(klass(), data, ['rmse', 'mae'], kf)
#  mean_rmse = '{:.3f}'.format(np.mean(out['test_rmse']))
#  mean_mae = '{:.3f}'.format(np.mean(out['test_mae']))
#  rmses.append(mean_rmse)
#  maes.append(mean_mae)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3596  0.3589  0.3580  0.3597  0.3598  0.3592  0.0007  
MAE (testset)     0.2696  0.2697  0.2690  0.2694  0.2694  0.2694  0.0002  
Fit time          43.45   43.71   42.61   43.05   44.87   43.54   0.76    
Test time         2.48    1.96    1.95    1.91    1.92    2.04    0.22    


{'test_rmse': array([0.35960331, 0.35886686, 0.35801519, 0.35971652, 0.35975558]),
 'test_mae': array([0.26963443, 0.26966146, 0.26900597, 0.2694467 , 0.2694098 ]),
 'fit_time': (43.45353817939758,
  43.71209931373596,
  42.605220079422,
  43.054938554763794,
  44.87299633026123),
 'test_time': (2.4776206016540527,
  1.9577667713165283,
  1.9497880935668945,
  1.9059031009674072,
  1.9178707599639893)}

In [96]:
# We'll try another algorithm
algo = NMF()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1622  0.1623  0.1621  0.1651  0.1615  0.1626  0.0013  
MAE (testset)     0.0897  0.0900  0.0895  0.0903  0.0895  0.0898  0.0003  
Fit time          55.11   54.70   53.81   54.06   56.52   54.84   0.96    
Test time         1.66    1.66    1.66    1.83    1.75    1.71    0.07    


{'test_rmse': array([0.16220675, 0.1623144 , 0.16206524, 0.16510892, 0.16152943]),
 'test_mae': array([0.08966708, 0.08997909, 0.08952882, 0.09025825, 0.0894568 ]),
 'fit_time': (55.105628967285156,
  54.70370602607727,
  53.81308627128601,
  54.0614275932312,
  56.52184438705444),
 'test_time': (1.664135456085205,
  1.656602144241333,
  1.664567470550537,
  1.8271098136901855,
  1.7473270893096924)}

In [97]:
# We'll try another algorithm
algo = CoClustering()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8156  0.7703  0.7039  0.8062  0.8115  0.7815  0.0420  
MAE (testset)     0.6199  0.5887  0.5441  0.6010  0.6206  0.5948  0.0281  
Fit time          25.82   25.43   25.89   25.48   25.51   25.63   0.19    
Test time         1.55    1.50    1.53    1.56    1.54    1.54    0.02    


{'test_rmse': array([0.81557869, 0.77034744, 0.70387169, 0.8062467 , 0.81149076]),
 'test_mae': array([0.61985597, 0.58873314, 0.54407056, 0.60095852, 0.62062552]),
 'fit_time': (25.815876483917236,
  25.430989503860474,
  25.88973903656006,
  25.4768967628479,
  25.512800931930542),
 'test_time': (1.5548427104949951,
  1.499992847442627,
  1.5329315662384033,
  1.5598266124725342,
  1.540849208831787)}

In [98]:
# We'll try another algorithm
algo = BaselineOnly()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7356  0.7335  0.7328  0.7342  0.7362  0.7345  0.0013  
MAE (testset)     0.6361  0.6345  0.6340  0.6354  0.6368  0.6354  0.0010  
Fit time          3.36    3.59    3.74    3.54    3.61    3.57    0.12    
Test time         1.40    1.42    1.41    1.46    1.40    1.42    0.02    


{'test_rmse': array([0.73557656, 0.7335376 , 0.73278829, 0.7341734 , 0.73619593]),
 'test_mae': array([0.63611612, 0.63449997, 0.63400093, 0.63542712, 0.63680474]),
 'fit_time': (3.363036632537842,
  3.59436297416687,
  3.740023136138916,
  3.543553352355957,
  3.6053879261016846),
 'test_time': (1.402249813079834,
  1.4162123203277588,
  1.4082386493682861,
  1.4551081657409668,
  1.3982295989990234)}

In [35]:
#################
# NO CBF MODELS #
#################

# drop unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['recommended_product_ids'])
orders_matrix.tail()

,Unnamed: 0,customer_id,product_id,score_detailed,review_id
112822,112645,b51593916b4b8e0d6f66f2ae24f2673d,4aa6014eceb682077f9dc4bffebc05b0,NaN,NaN
112823,112646,84c5d4fbaf120aae381fad077416eaa0,32e07fd915822b0765e448c4dd74c828,NaN,NaN
112824,112647,29309aa813182aaddc9b259e31b870e6,72a30483855e2eafc67aee5dc2560482,NaN,NaN
112825,112648,b5e6afd5a41800fdf401e0272ca74655,9c422a519119dcad7575db5af1ba540e,5.0,fba117c9ac40d41ca7be54741f471303
112826,112649,96d649da0cc4ff33bb408b199d4c7dcf,350688d9dc1e75ff97be326363655e01,NaN,NaN


In [37]:
# drop unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['review_id'])
orders_matrix.head()

,Unnamed: 0,customer_id,product_id,score_detailed
0,0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000
1,1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN
2,2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000
3,3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN
4,4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704


In [38]:
# drop unnecesary columns

orders_matrix = orders_matrix.drop(columns = ['Unnamed: 0'])
orders_matrix.head()

,customer_id,product_id,score_detailed
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,5.000
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,NaN
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5.000
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,NaN
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,4.704


In [39]:
orders_matrix = orders_matrix.dropna(subset = ['score_detailed'])

In [41]:
#################
# NO CBF MODELS #
#################

# set dataframe for Surprise models

reader = Reader(line_format='user item rating', sep=';', rating_scale=(0, 6), skip_lines=1)
data = Dataset.load_from_df(orders_matrix, reader = reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5465  1.5343  1.5366  1.5432  1.5402  1.5402  0.0044  
MAE (testset)     1.3559  1.3393  1.3435  1.3500  1.3497  1.3477  0.0058  
Fit time          2.11    2.41    2.31    2.31    2.14    2.25    0.11    
Test time         0.06    0.06    0.05    0.06    0.06    0.06    0.00    


{'test_rmse': array([1.54650978, 1.53432188, 1.53663456, 1.54316513, 1.54022104]),
 'test_mae': array([1.35594341, 1.33927312, 1.34346571, 1.35003594, 1.34973281]),
 'fit_time': (2.1102702617645264,
  2.4065635204315186,
  2.3098227977752686,
  2.305830955505371,
  2.1402745246887207),
 'test_time': (0.06339383125305176,
  0.05767703056335449,
  0.05488157272338867,
  0.0608370304107666,
  0.06283354759216309)}

In [42]:
#################
# NO CBF MODELS #
#################

# We'll try another algorithm
algo = NMF()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4935  1.4833  1.5001  1.4930  1.4821  1.4904  0.0068  
MAE (testset)     1.2399  1.2313  1.2483  1.2373  1.2296  1.2373  0.0067  
Fit time          6.20    5.64    6.03    5.73    5.95    5.91    0.20    
Test time         0.06    0.05    0.05    0.05    0.05    0.05    0.00    


{'test_rmse': array([1.49353026, 1.48328655, 1.50014326, 1.49303943, 1.48209702]),
 'test_mae': array([1.23989456, 1.23131686, 1.24831026, 1.23726269, 1.22962723]),
 'fit_time': (6.195697069168091,
  5.636892795562744,
  6.030869245529175,
  5.725656270980835,
  5.949063062667847),
 'test_time': (0.0578463077545166,
  0.05485391616821289,
  0.05089592933654785,
  0.050864219665527344,
  0.05385756492614746)}

In [43]:
#################
# NO CBF MODELS #
#################

# We'll try another algorithm
algo = CoClustering()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5319  1.5872  1.5469  1.5207  1.5645  1.5502  0.0236  
MAE (testset)     1.3413  1.4191  1.3567  1.3134  1.3815  1.3624  0.0359  
Fit time          6.37    7.21    5.73    5.23    5.24    5.96    0.75    
Test time         0.05    0.05    0.04    0.05    0.05    0.05    0.00    


{'test_rmse': array([1.53190153, 1.58719991, 1.54688051, 1.52073767, 1.56449781]),
 'test_mae': array([1.34132907, 1.41911158, 1.35665779, 1.31342821, 1.38152053]),
 'fit_time': (6.366780996322632,
  7.212711811065674,
  5.733665704727173,
  5.230978488922119,
  5.236996412277222),
 'test_time': (0.0468745231628418,
  0.04690694808959961,
  0.04291653633117676,
  0.04886889457702637,
  0.04884004592895508)}

In [44]:
#################
# NO CBF MODELS #
#################

# We'll try another algorithm
algo = BaselineOnly()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6186  1.6025  1.6189  1.6075  1.5922  1.6080  0.0101  
MAE (testset)     1.4403  1.4294  1.4407  1.4361  1.4231  1.4339  0.0068  
Fit time          0.18    0.24    0.19    0.18    0.18    0.19    0.02    
Test time         0.05    0.05    0.05    0.05    0.05    0.05    0.00    


{'test_rmse': array([1.61858336, 1.60245909, 1.61894515, 1.60754582, 1.59223555]),
 'test_mae': array([1.44034939, 1.42938179, 1.44069647, 1.43608906, 1.42309005]),
 'fit_time': (0.1795482635498047,
  0.2373645305633545,
  0.18849420547485352,
  0.17852210998535156,
  0.18447566032409668),
 'test_time': (0.05083346366882324,
  0.04887795448303223,
  0.04983663558959961,
  0.04986739158630371,
  0.047873497009277344)}